# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the SQL questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create code to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- Return the text of the reviews for the most reviewed restaurant. 
- Return the name of the business with the most recent review. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


## Part 1 - Understanding your data and question

Lok at the question and determine what data you will need to store in your database in order to answer the questions. Start to thin about your DB schema. What tables will you want to create and what columns will you ahve for those tables. 

Look at the API documentation, and determine what fields of the API response you will match up with the columns you want in your DB tables. 


https://www.yelp.com/developers/documentation/v3/get_started

## Part 2 - Set up the DB

Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

### Foriegn Keys

For this project you will have two tables that you need to link using a foreign key. Below is some information about how to create a foriegn key.  

http://www.mysqltutorial.org/mysql-foreign-key/



### Using DB:
    
For this lab, you can either store the data on one DB or put in on both of the partners DBs. If you decide to put it on one DB, you want to make sure both partners have access to it.  To do this you want to add a user to your DB.  

[how to add a new user](https://howchoo.com/g/mtm3zdq2nzv/how-to-add-a-mysql-user-and-grant-privileges)

In [ ]:
## Connect to DB server on AWS

In [ ]:
## Create new DB 

In [ ]:
# Create a table for the Businesses

In [ ]:
# Create a table for the reviews

## Part 3 - Create ETL pipeline for the business data from the API

Now that you know what data you need from the API, you want to write code that will execute a api call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*api call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

In [1]:
import requests
import json
client_id = 'Vyfh0fZiTSVYH8YyPfD54g' #Your client ID goes here (as a string)
api_key = 'HmydvxPN1s-g-6LwnD1lDVAkXXLrbzk3iwCrmV9yDtwAXpRtARyWXReYCq9kTTEERgNQsrE5WxcSZdZj-r28tUhPBlOqwr7Tbj7qBv7wzoc1VnmcnGA6tEr7of4pXnYx' #Your api key goes here (as a string)

In [2]:
term = 'pub'
location = 'NY'
SEARCH_LIMIT = 50

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT
            }
response = requests.get(url, headers=headers, params=url_params)
print(response)
print(type(response.text))
print(response.text)

<Response [200]>
<class 'str'>
{"businesses": [{"id": "OxqUWRZFUOz3r6_4bh4mfg", "alias": "cock-and-bull-new-york", "name": "Cock & Bull", "image_url": "https://s3-media2.fl.yelpcdn.com/bphoto/-82MfQUN7FXel19Ra9BsLg/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/cock-and-bull-new-york?adjust_creative=Vyfh0fZiTSVYH8YyPfD54g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=Vyfh0fZiTSVYH8YyPfD54g", "review_count": 532, "categories": [{"alias": "british", "title": "British"}, {"alias": "pubs", "title": "Pubs"}, {"alias": "gastropubs", "title": "Gastropubs"}], "rating": 4.0, "coordinates": {"latitude": 40.7560908823673, "longitude": -73.9804858350908}, "transactions": ["restaurant_reservation"], "price": "$$", "location": {"address1": "23 W 45th St", "address2": null, "address3": "", "city": "New York", "zip_code": "10036", "country": "US", "state": "NY", "display_address": ["23 W 45th St", "New York, NY 10036"]}, "phone": "+12128191900", "display_phone": "

In [7]:
import mysql.connector
from mysql.connector import errorcode
cnx = mysql.connector.connect(
    host = 'database-1.c6dr0kuue1ke.us-east-2.rds.amazonaws.com',
    user = 'admin',
    passwd = 'Alex27138231',
    database = 'yelp'
)
cnx
cursor = cnx.cursor()

In [8]:
print(cnx)

In [9]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

In [10]:
db_name = 'yelp'
create_database(cursor, db_name)

Failed creating database: 1007 (HY000): Can't create database 'yelp'; database exists


In [11]:
# db_name = 'yelp'
TABLES = {}
TABLES['Businesses'] = (
    "CREATE TABLE Businesses ("
    "  id varchar(50),"
    "  name varchar(100),"
    "  price varchar(6),"
    "  rating float(6),"
    "  review_count int(10)"
    ") ENGINE=InnoDB")

In [12]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

Creating table Businesses: already exists.


In [13]:
response.json().keys()

dict_keys(['businesses', 'total', 'region'])

In [14]:
for key in response.json().keys():
    print(key)
    value = response.json()[key] #Use standard dictionary formatting
    print(type(value)) #What type is it?
    print('\n\n') #Seperate out data

businesses
<class 'list'>



total
<class 'int'>



region
<class 'dict'>





In [15]:
yelp_data =response.json()
yelp_data['businesses']

[{'id': 'OxqUWRZFUOz3r6_4bh4mfg',
  'alias': 'cock-and-bull-new-york',
  'name': 'Cock & Bull',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/-82MfQUN7FXel19Ra9BsLg/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/cock-and-bull-new-york?adjust_creative=Vyfh0fZiTSVYH8YyPfD54g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=Vyfh0fZiTSVYH8YyPfD54g',
  'review_count': 532,
  'categories': [{'alias': 'british', 'title': 'British'},
   {'alias': 'pubs', 'title': 'Pubs'},
   {'alias': 'gastropubs', 'title': 'Gastropubs'}],
  'rating': 4.0,
  'coordinates': {'latitude': 40.7560908823673,
   'longitude': -73.9804858350908},
  'transactions': ['restaurant_reservation'],
  'price': '$$',
  'location': {'address1': '23 W 45th St',
   'address2': None,
   'address3': '',
   'city': 'New York',
   'zip_code': '10036',
   'country': 'US',
   'state': 'NY',
   'display_address': ['23 W 45th St', 'New York, NY 10036']},
  'phone': '+12128191900',
  'displ

In [16]:
import pandas as pd
pd.DataFrame(yelp_data['businesses']).head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,OxqUWRZFUOz3r6_4bh4mfg,cock-and-bull-new-york,Cock & Bull,https://s3-media2.fl.yelpcdn.com/bphoto/-82MfQ...,False,https://www.yelp.com/biz/cock-and-bull-new-yor...,532,"[{'alias': 'british', 'title': 'British'}, {'a...",4.0,"{'latitude': 40.7560908823673, 'longitude': -7...",[restaurant_reservation],$$,"{'address1': '23 W 45th St', 'address2': None,...",+12128191900,(212) 819-1900,5750.495668
1,3NbhYebjZPjLTWZZyRuhsw,the-winslow-new-york-2,The Winslow,https://s3-media2.fl.yelpcdn.com/bphoto/KqK_JD...,False,https://www.yelp.com/biz/the-winslow-new-york-...,529,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",4.5,"{'latitude': 40.7327766418457, 'longitude': -7...",[restaurant_reservation],$$,"{'address1': '243 E 14th St', 'address2': '', ...",+12127777717,(212) 777-7717,3132.702360
2,ZnhRUBos9FGQZrzbcPn6Vg,o-haras-restaurant-and-pub-new-york,O'Hara's Restaurant and Pub,https://s3-media2.fl.yelpcdn.com/bphoto/nHPOfx...,False,https://www.yelp.com/biz/o-haras-restaurant-an...,460,"[{'alias': 'irish_pubs', 'title': 'Irish Pub'}...",4.0,"{'latitude': 40.7093473589949, 'longitude': -7...",[],$$,"{'address1': '120 Cedar St', 'address2': 'Ste ...",+12122673032,(212) 267-3032,1575.039638
3,bkIMbvXOcztDVz8bpyhMzQ,the-dead-poet-new-york,The Dead Poet,https://s3-media4.fl.yelpcdn.com/bphoto/fW9WcV...,False,https://www.yelp.com/biz/the-dead-poet-new-yor...,424,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 40.78499, 'longitude': -73.97735}",[],$$,"{'address1': '450 Amsterdam Ave', 'address2': ...",+12125955670,(212) 595-5670,8959.249622
4,RemvTpxalDjSew5HwmNzUQ,the-dead-rabbit-new-york,The Dead Rabbit,https://s3-media2.fl.yelpcdn.com/bphoto/_4T5R0...,False,https://www.yelp.com/biz/the-dead-rabbit-new-y...,1558,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.0,"{'latitude': 40.70329, 'longitude': -74.01102}",[],$$,"{'address1': '30 Water St', 'address2': '', 'a...",+16464227906,(646) 422-7906,1431.039986


In [17]:
def parsed_tuples(data):
    parsed_tuple_list = []
    for i in yelp_data['businesses']:
        if 'id' in i:
            pass
        else:
            i['id']='nan'
        if 'name' in i:
            pass
        else:
            i['name']='nan'
        if 'review_count' in i:
            pass
        else:
            i['review_count']='nan'
        if 'rating' in i:
            pass
        else:
            i['rating']='nan'
        if 'price' in i:
            pass
        else:
            i['price']='nan'
        parsed_tuple_list.append(tuple((i['id'], i['name'], i['review_count'], i['rating'], i['price'])))
    return parsed_tuple_list

aa = parsed_tuples(yelp_data)
aa

[('OxqUWRZFUOz3r6_4bh4mfg', 'Cock & Bull', 532, 4.0, '$$'),
 ('3NbhYebjZPjLTWZZyRuhsw', 'The Winslow', 529, 4.5, '$$'),
 ('ZnhRUBos9FGQZrzbcPn6Vg', "O'Hara's Restaurant and Pub", 460, 4.0, '$$'),
 ('bkIMbvXOcztDVz8bpyhMzQ', 'The Dead Poet', 424, 4.0, '$$'),
 ('RemvTpxalDjSew5HwmNzUQ', 'The Dead Rabbit', 1558, 4.0, '$$'),
 ('QArou0r7CmLNPWYRA9ecog', 'McSorleys Old Ale House', 1758, 4.0, '$'),
 ('c3Ww3pgeHjBi3ctkeUWrjQ', 'Whiskey Tavern', 781, 4.5, '$$'),
 ('lHGQM__w1x5Dxiq3aRPCTQ',
  'White Horse Tavern - Financial District',
  368,
  4.0,
  '$'),
 ('1pMep2AS_MPdobWwDzgkPw', '4th Avenue Pub', 306, 4.0, '$$'),
 ('PMp7WlHIXQUnej-Ds1jRfw', 'Blind Tiger Ale House', 883, 4.0, '$$'),
 ('vdrHSnRc1X2pJiLJuiOfGA', 'Toad Hall', 100, 4.0, '$$'),
 ('7boggcCID7gVdNFQn4e8hQ', "Carragher's Pub & Restaurant", 224, 4.0, '$$'),
 ('FgsN_18sk15FcTnYY3Ghjg', 'The Irish American', 291, 4.0, '$$'),
 ('kMxXLrASxsbmS-ZzAaBe-A', "Molly's", 505, 4.0, '$$'),
 ('xHgMfdyLA5SPQz4c3ItTgg', 'The Commissioner', 133, 5.0

In [18]:
#create the insert statment
stmt = "INSERT INTO Businesses (id, name, review_count, rating, price) VALUES (%s, %s, %s, %s, %s)"

In [19]:
#insert all of the records and commit it
cursor.executemany(stmt, aa)
cnx.commit()

In [20]:
def yelp_call():
    term = 'pub'
    location = 'NY'
    SEARCH_LIMIT = 50

    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
            'Authorization': 'Bearer {}'.format(api_key),
        }

    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'limit': SEARCH_LIMIT
                }
    response = requests.get(url, headers=headers, params=url_params)
#     print(response)
#     print(type(response.text))
#     print(response.text)
    return response.json()


In [21]:
test = yelp_call()

In [22]:
# test

In [23]:
# parsed_tuples(test)

In [24]:
def db_insert(parsed_tuples):
    stmt = "INSERT INTO Businesses (id, name, review_count, rating, price) VALUES (%s, %s, %s, %s, %s)"
    cursor.executemany(stmt, parsed_tuples)
    cnx.commit()

In [25]:
# len(parsed_results)

In [26]:
cur = 0
parsed_list = []
#set up a while loop to go through and grab the result 
while cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call()
    #after you get your results you can now use your function to parse those results
    parsed_results = parsed_tuples(results)
    parsed_list.extend(parsed_results)
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 50

In [27]:
for i in parsed_list:
    print(i[0])

OxqUWRZFUOz3r6_4bh4mfg
3NbhYebjZPjLTWZZyRuhsw
ZnhRUBos9FGQZrzbcPn6Vg
bkIMbvXOcztDVz8bpyhMzQ
RemvTpxalDjSew5HwmNzUQ
QArou0r7CmLNPWYRA9ecog
c3Ww3pgeHjBi3ctkeUWrjQ
lHGQM__w1x5Dxiq3aRPCTQ
1pMep2AS_MPdobWwDzgkPw
PMp7WlHIXQUnej-Ds1jRfw
vdrHSnRc1X2pJiLJuiOfGA
7boggcCID7gVdNFQn4e8hQ
FgsN_18sk15FcTnYY3Ghjg
kMxXLrASxsbmS-ZzAaBe-A
xHgMfdyLA5SPQz4c3ItTgg
7sQYI_Ym9RzlxHIhJpj6hg
dzlhiEg7nE49iReUC10w_A
K21NqbD_aa2FDUmQkKRltQ
1hScmfYV42XshcprL79MAQ
3ESlPTtjsEsyI5jhJFvBbw
IGZkGbwE36iE1yaDW5NNKA
zjoHov9SWVHHUk_2zSNzPA
ZCfqAtpASbibf_YaU1rIpw
lqkqsHe_2KN35LE5JIaYNg
mVN2G1c5bRtAsNlr2Tbx6Q
veku2cg2qaV-7MxwhOMBmg
IBgjj5VBu5JJvKgo9EzKrg
VNee09tB2NVGTe3g1tFylw
Gey_giAxXSzPbpVigXV9jQ
NPG2jp4C61RJbmbA35p6xA
LDwxXfapMw6xNbTF0YDFaQ
yFB5JUiz-SPpnhWg2iCU9w
GW42yOQB6G97Pc0KujI3lA
f7i6LxdwbDXExDn_58tXaA
Xu8H24NxJJxEE-9wGp_AMA
RBkz0yuQfs_5ExPhL-6jpg
sDajjybHSYFw8gTVbknPGA
Z65cd3Z7JvwK8TnSVg-cfA
0yg9DS4RDceQ6HgH3Jx80Q
HWTH6QwatW6lcYaIWgw4Kg
6hKJePanmG9AxK4dfwgyyg
cBC55mBdY_1Zj7dfVeWWQg
ohLIPmzTysowInbte9XhMw
H3AnaDvXg_t

In [29]:
cursor.execute('TRUNCATE TABLE Businesses')

In [ ]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB

In [ ]:
# Write a function to take your parsed data and insert it into the DB

In [ ]:
# Write a script that combines the three functions above into a single process.

## Part 4 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews.

In [ ]:
# write a SQL query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

In [ ]:
# write a function that takes a business id 
# and makes a call to the API for reivews


In [ ]:
# Write a function to parse out the relevant information from the reviews

In [ ]:
# write a function to insert the parsed data into the reviews table

In [ ]:
# combine the functions above into a single script  

## Part 5 -  Write SQL queries that will answer the questions posed. 

Now that your data is in the DB, you can answer the questions. Write SQL querires that will answer the questions. **Do not use Pandas in this lab**

In [ ]:
# create connection

In [ ]:
# execute SQL queries

# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
def yelp_call(url_params, api_key):
    # your code to make the yelp call
    return data

In [ ]:
def parse_results(results):
    # your code to parse the result to make them easier to insert into the DB
    return parsed_results

In [ ]:
def db_insert(cnx, cursor, parsed results):
    # your code to insert and commit the results
    # I would create the connection and cursor outside of this function and then pass it through

In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
for cur in range(0, 1000, 50)
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 50

In [130]:
review_id=[]
for i in parsed_list:
    review_id.append(i[0])
review_id

['OxqUWRZFUOz3r6_4bh4mfg',
 '3NbhYebjZPjLTWZZyRuhsw',
 'ZnhRUBos9FGQZrzbcPn6Vg',
 'bkIMbvXOcztDVz8bpyhMzQ',
 'RemvTpxalDjSew5HwmNzUQ',
 'QArou0r7CmLNPWYRA9ecog',
 'c3Ww3pgeHjBi3ctkeUWrjQ',
 'lHGQM__w1x5Dxiq3aRPCTQ',
 '1pMep2AS_MPdobWwDzgkPw',
 'PMp7WlHIXQUnej-Ds1jRfw',
 'vdrHSnRc1X2pJiLJuiOfGA',
 '7boggcCID7gVdNFQn4e8hQ',
 'FgsN_18sk15FcTnYY3Ghjg',
 'kMxXLrASxsbmS-ZzAaBe-A',
 'xHgMfdyLA5SPQz4c3ItTgg',
 '7sQYI_Ym9RzlxHIhJpj6hg',
 'dzlhiEg7nE49iReUC10w_A',
 'K21NqbD_aa2FDUmQkKRltQ',
 '1hScmfYV42XshcprL79MAQ',
 '3ESlPTtjsEsyI5jhJFvBbw',
 'IGZkGbwE36iE1yaDW5NNKA',
 'zjoHov9SWVHHUk_2zSNzPA',
 'ZCfqAtpASbibf_YaU1rIpw',
 'lqkqsHe_2KN35LE5JIaYNg',
 'mVN2G1c5bRtAsNlr2Tbx6Q',
 'veku2cg2qaV-7MxwhOMBmg',
 'IBgjj5VBu5JJvKgo9EzKrg',
 'VNee09tB2NVGTe3g1tFylw',
 'Gey_giAxXSzPbpVigXV9jQ',
 'NPG2jp4C61RJbmbA35p6xA',
 'LDwxXfapMw6xNbTF0YDFaQ',
 'yFB5JUiz-SPpnhWg2iCU9w',
 'GW42yOQB6G97Pc0KujI3lA',
 'f7i6LxdwbDXExDn_58tXaA',
 'Xu8H24NxJJxEE-9wGp_AMA',
 'RBkz0yuQfs_5ExPhL-6jpg',
 'sDajjybHSYFw8gTVbknPGA',
 

In [62]:
def yelp_review_call(pub_ids):
    url1 = 'https://api.yelp.com/v3/businesses/'
    url2='/reviews'
    return {pub_ids:requests.get(url1+pub_ids+url2, headers=headers).json()}    

In [65]:
reviews=[]
for i in review_id:
    reviews.append(yelp_review_call(i))

In [201]:
reviews_data=reviews[0:809]

In [222]:
reviews_data[0]['OxqUWRZFUOz3r6_4bh4mfg']['reviews'][0]['text']

'I absolutely loved this place beacuse they offered delicious vegan food. Not just a salad and fries, these guys had cauliflower bites, vegan wings, and pot...'

In [220]:
len(review_data[0][review_id[0]]['reviews'])

3

In [209]:
review_text=[]

for i in range(0,len(review_data)):
    for j in range(0,len(review_data[i][review_id[i]]['reviews'])):
        review_text.append(review_data[i][review_id[i]]['reviews'][j]['text'])
        

KeyError: 'reviews'

In [188]:
review_data[0:880][879][review_id[879]]

{'error': {'code': 'ACCESS_LIMIT_REACHED',
  'description': "You've reached the access limit for this client. See instructions for requesting a higher access limit at https://www.yelp.com/developers/documentation/v3/rate_limiting"}}

In [244]:
review_text=[]
for i in range(0,len(reviews_data)):
    for j in review_data[i][review_id[i]]['reviews'] :           
            if 'reviews' in review_data[i][review_id[i]].keys():
                review_text.append(j['text'])
                
            else:
                None 
                review_text.append(j['text'])
            

In [245]:
review_text

['I absolutely loved this place beacuse they offered delicious vegan food. Not just a salad and fries, these guys had cauliflower bites, vegan wings, and pot...',
 "Cottage pie was tasty and satisfying -Yummy! Didn't love the mashed pea side which could use some seasoning. \n\nMy husband got the fish and chips which were...",
 'I stopped here for a quick drink before heading downtown for an event. I was with five other people in midtown around 5:30, so you can imagine it was...',
 'I held an event at the Winslow for Halloween and I only have good things to say about the drinks, the atmosphere and the management/ staff.\n\nThe space was...',
 'Great service. First time here. I picked a spot for an early dinner before a Rangers game in Jersey, ugh. I know, right. \n\nOrdered for both of us....',
 'After visiting several different spaces to determine which would work best for a client appreciation event, we ultimately went with the Winslow, which was...',
 "This is my first time to visit 

In [224]:
reviews_data[0]['OxqUWRZFUOz3r6_4bh4mfg']['reviews'][0]

{'id': 'VNV4Io4-EmVGDx5fyOaMjg',
 'url': 'https://www.yelp.com/biz/cock-and-bull-new-york?adjust_creative=Vyfh0fZiTSVYH8YyPfD54g&hrid=VNV4Io4-EmVGDx5fyOaMjg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=Vyfh0fZiTSVYH8YyPfD54g',
 'text': 'I absolutely loved this place beacuse they offered delicious vegan food. Not just a salad and fries, these guys had cauliflower bites, vegan wings, and pot...',
 'rating': 5,
 'time_created': '2019-11-22 20:41:29',
 'user': {'id': 'NxsXhyK8O0wUcD5jtyC4YA',
  'profile_url': 'https://www.yelp.com/user_details?userid=NxsXhyK8O0wUcD5jtyC4YA',
  'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/oa1UU3AYA4zu85DzaDVFLg/o.jpg',
  'name': 'Jen M.'}}

In [231]:
review_rating=[]
for i in range(0,len(reviews_data)):
    for j in reviews_data[i][review_id[i]]['reviews'] :           
            if 'reviews' in review_data[i][review_id[i]].keys():
                review_rating.append(j['rating'])
                
            else:
                None 
                review_rating.append(j['rating'])

In [232]:
review_rating

[5,
 4,
 4,
 5,
 4,
 5,
 5,
 1,
 2,
 4,
 4,
 4,
 5,
 4,
 4,
 5,
 4,
 4,
 5,
 5,
 5,
 4,
 1,
 3,
 5,
 3,
 5,
 5,
 3,
 3,
 5,
 3,
 1,
 5,
 2,
 2,
 4,
 5,
 4,
 5,
 2,
 1,
 5,
 4,
 4,
 5,
 4,
 4,
 4,
 5,
 3,
 5,
 1,
 3,
 4,
 5,
 4,
 4,
 3,
 5,
 5,
 4,
 4,
 5,
 5,
 5,
 5,
 4,
 5,
 4,
 4,
 1,
 5,
 4,
 4,
 5,
 2,
 3,
 5,
 1,
 4,
 4,
 3,
 3,
 5,
 5,
 3,
 5,
 3,
 3,
 4,
 1,
 5,
 5,
 4,
 5,
 5,
 4,
 5,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 3,
 2,
 4,
 1,
 4,
 5,
 3,
 5,
 5,
 3,
 1,
 5,
 4,
 5,
 5,
 4,
 4,
 5,
 5,
 4,
 5,
 3,
 3,
 5,
 5,
 2,
 5,
 4,
 4,
 5,
 3,
 4,
 5,
 4,
 4,
 5,
 1,
 5,
 5,
 5,
 5,
 5,
 4,
 5,
 5,
 4,
 4,
 5,
 4,
 5,
 5,
 1,
 2,
 4,
 4,
 4,
 5,
 4,
 4,
 5,
 4,
 4,
 5,
 5,
 5,
 4,
 1,
 3,
 5,
 3,
 5,
 5,
 3,
 3,
 5,
 3,
 1,
 5,
 2,
 2,
 4,
 5,
 4,
 5,
 2,
 1,
 5,
 4,
 4,
 5,
 4,
 4,
 4,
 5,
 3,
 5,
 1,
 3,
 4,
 5,
 4,
 4,
 3,
 5,
 5,
 4,
 4,
 5,
 5,
 5,
 5,
 4,
 5,
 4,
 4,
 1,
 5,
 4,
 4,
 5,
 2,
 3,
 5,
 1,
 4,
 4,
 3,
 3,
 5,
 5,
 3,
 5,
 3,
 3,
 4,
 1,
 5,
 5,
 4,
 5,
 5,
 4,
 5,
 4,


In [250]:
review_time_created=[]
for i in range(0,len(reviews_data)):
    for j in reviews_data[i][review_id[i]]['reviews'] :           
            if 'reviews' in review_data[i][review_id[i]].keys():
                review_time_created.append(j['time_created'])
                
            else:
                None 
                review_time_created.append(j['time_created'])

In [251]:
review_time_created

['2019-11-22 20:41:29',
 '2020-01-27 06:25:43',
 '2019-12-21 17:06:35',
 '2019-11-20 10:27:58',
 '2020-01-22 15:39:05',
 '2019-11-18 12:55:42',
 '2019-12-27 13:02:56',
 '2019-12-15 18:53:59',
 '2019-11-11 08:28:51',
 '2020-01-08 11:16:18',
 '2019-09-28 06:25:15',
 '2019-09-07 18:20:24',
 '2020-01-12 08:07:35',
 '2020-01-22 15:28:44',
 '2020-01-20 12:52:28',
 '2019-11-27 08:57:10',
 '2020-01-19 07:48:45',
 '2019-12-16 21:37:48',
 '2020-01-27 13:15:17',
 '2019-12-28 20:22:07',
 '2019-11-30 16:28:03',
 '2019-12-17 03:16:14',
 '2019-12-28 10:45:23',
 '2019-12-20 18:52:16',
 '2019-09-27 12:29:36',
 '2019-12-10 13:13:04',
 '2019-04-30 19:20:32',
 '2020-01-27 03:43:38',
 '2019-12-26 09:03:43',
 '2019-07-06 08:56:38',
 '2020-01-28 06:03:32',
 '2019-12-15 18:49:47',
 '2019-12-14 20:08:44',
 '2019-08-14 14:57:36',
 '2019-12-21 04:12:14',
 '2019-09-29 18:03:20',
 '2020-01-18 16:59:58',
 '2019-12-30 04:10:03',
 '2019-12-06 08:49:09',
 '2019-12-20 16:12:37',
 '2019-11-24 05:07:24',
 '2019-12-12 09:

In [237]:
TABLES = {}
TABLES['Reviews'] = (
    "CREATE TABLE Reviews ("
    "  id varchar(50),"
    "  rating float(6),"
    "  text varchar(1000)"
    ") ENGINE=InnoDB")

In [238]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

Creating table Reviews: Lost connection to MySQL server at 'database-1.c6dr0kuue1ke.us-east-2.rds.amazonaws.com:3306', system error: 32 Broken pipe


In [248]:
def parsed_tuples2(data):
    parsed_tuple_list2 = []
    for i in yelp_data['reviews_data']:
        if 'id' in i:
            pass
        else:
            i['id']='nan'
        if 'rating' in i:
            pass
        else:
            i['rating']='nan'
        if 'text' in i:
            pass
        else:
            i['text']='nan'
        parsed_tuple_list.append(tuple((i['id'], i['rating'], i['text'])))
    return parsed_tuple_list2

bb = parsed_tuples2(yelp_data)
bb

KeyError: 'reviews_data'

In [ ]:
def db_insert2(parsed_tuples):
    stmt = "INSERT INTO Reviews (id, rating, text) VALUES (%s, %s, %s)"
    cursor.executemany(stmt, parsed_tuples)
    cnx.commit()

In [ ]:
cur = 0

#set up a while loop to go through and grab the result 
while cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_review_call('pub_ids')
    #after you get your results you can now use your function to parse those results
    parsed_results = parsed_tuples(results)
    
    # use your function to insert your parsed results into the db
    db_insert2(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 50